# Run Analysis

Data sources are stored as arbitrarily deep directories in /data. A data source is a directory with an index file which contains front matter such as filetype, cadence, etc related to how to access the data and when/whether to update the data automatically.  

Some data sources have a static cadence, meaning they wont be automatically updated by this notebook.  

Others specify an update cadence, a time last updated, and a method of updating which may include things like an api key, etc.  

An analysis, likewise will be an arbitrarily deep sirectory wihtin /analysis which contains an index file with front matter like title and dependencies.  

Dependencies are paths to data sources or other analyses. Whenever an analysis was last modified before any one of its dependencies, the analysis is stale and needs to be run again.  

In [3]:
#### Make sure all required packages are installed and imported

import importlib
import subprocess
import sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None, required: bool = True):
    """Import a package, installing it if missing."""
    try:
        return importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
        except Exception:
            if required:
                raise
    mod = importlib.import_module(import_name or pkg_name)
    globals()[import_name or pkg_name] = mod
    return mod

_ensure('pandas')
_ensure('requests')
_ensure('feedparser')
_ensure('textblob')
_ensure('pyyaml', 'yaml')
_ensure('jupyter', required=False)
_ensure('nbconvert', required=False)
print('All dependencies ready.')


All dependencies ready.


## Update Data Sources

This cell needs to find all of the /data index files, check whether that data source needs to be updated, and then do whatever updates are appropriate.  



In [4]:
from pathlib import Path
import datetime as dt
import os
import re
import shutil
import json
import feedparser
import textblob
import pandas as pd
import requests
import urllib.parse
import yaml
from typing import Optional

CADENCE_SECONDS = {
    'hourly': 3600,
    'daily': 86400,
    'weekly': 604800,
    'monthly': 2592000,
    'quarterly': 7776000,
}

def substitute_date_tokens(url: str) -> str:
    """Replace [date %Y-%m-%d] tokens with today's date."""
    def _replace(m):
        return dt.date.today().strftime(m.group(1).strip())
    return re.sub(r"\[date\s+([^\]]+)\]", _replace, url)

def add_apikey(url: str, env_var: Optional[str]) -> str:
    """Append an API key from the environment if available."""
    if env_var:
        key = os.getenv(env_var)
        if key:
            sep = '&' if '?' in url else '?'
            return f"{url}{sep}api_key={urllib.parse.quote_plus(key)}"
    return url

def _parse_meta(path: Path):
    """Return metadata and body text from a markdown file."""
    text = path.read_text()
    m = re.search(r'^---\n(.*?)\n---\n?', text, re.S)
    if m:
        try:
            meta = yaml.safe_load(m.group(1)) or {}
        except Exception as e:
            print(f'Error parsing metadata in {path}:', e)
            raise
        body = text[m.end():]
    else:
        meta, body = {}, text
    return meta, body

def _write_meta(path: Path, meta: dict, body: str):
    """Write metadata and body back to a markdown file."""
    header = yaml.safe_dump(meta, sort_keys=False).strip()
    path.write_text(f"---\n{header}\n---\n{body}")

def _read_index(idx_file: Path):
    meta, body = _parse_meta(idx_file)
    url = str(meta.get('url', '')).strip()
    if not url or url.lower() in ('n/a', 'na', 'none'):
        return None
    filetype = str(meta.get('filetype', '')).strip().lstrip('.')
    cadence = str(meta.get('cadence', 'monthly')).lower()
    api_key = meta.get('api_key')
    last_fetch = (
        pd.to_datetime(meta.get('last_fetched'))
        if meta.get('last_fetched') else None
    )
    return meta, body, url, filetype, cadence, api_key, last_fetch

def _build_paths(idx_file: Path, filetype: str, cadence: str,
                 now: dt.datetime, today: dt.date):
    folder = idx_file.parent
    output_ext = 'json' if filetype in ('rss', 'xml') else filetype
    latest_fp = folder / f'latest.{output_ext}'
    name = f"{now:%Y-%m-%d-%H}" if cadence == 'hourly' else f"{today:%Y-%m-%d}"
    dated_fp = folder / f'{name}.{output_ext}'
    return folder, latest_fp, dated_fp

def _should_skip(latest_fp: Path, dated_fp: Path,
                 last_fetch, now: dt.datetime, min_age: int) -> bool:
    if latest_fp.exists() and last_fetch and (now - last_fetch).total_seconds() < min_age:
        if dated_fp.exists() and latest_fp.read_bytes() != dated_fp.read_bytes():
            shutil.copyfile(dated_fp, latest_fp)
        return True
    return False

def _fetch_content(req_url: str, filetype: str) -> bytes:
    """Return bytes from a URL or an empty result on error."""
    try:
        resp = requests.get(
            req_url,
            timeout=30,
            headers={'User-Agent': 'Mozilla/5.0'},
        )
        resp.raise_for_status()
    except requests.RequestException as exc:
        print(f"Request failed for {req_url}: {exc}")
        return b''
    if filetype in ('rss', 'xml'):
        feed = feedparser.parse(resp.content)
        entries = []
        for entry in feed.entries:
            text = ' '.join(
                filter(None, [entry.get('title'), entry.get('summary')])
            )
            pol = textblob.TextBlob(text).sentiment.polarity
            entries.append(
                {
                    'title': entry.get('title'),
                    'link': entry.get('link'),
                    'published': entry.get('published'),
                    'sentiment': pol,
                }
            )
        json_bytes = json.dumps({'entries': entries}, ensure_ascii=False, indent=2).encode('utf-8')
        return json_bytes
    return resp.content

def _write_files(content: bytes, dated_fp: Path, latest_fp: Path):
    dated_fp.write_bytes(content)
    shutil.copyfile(dated_fp, latest_fp)

def _process_index(idx_file: Path, base: Path,
                   now: dt.datetime, today: dt.date) -> str | None:
    result = _read_index(idx_file)
    if not result:
        return None
    meta, body, url, filetype, cadence, api_key, last_fetch = result
    min_age = CADENCE_SECONDS.get(cadence, 2592000)
    folder, latest_fp, dated_fp = _build_paths(idx_file, filetype, cadence, now, today)
    if _should_skip(latest_fp, dated_fp, last_fetch, now, min_age):
        return None
    req_url = add_apikey(substitute_date_tokens(url), api_key)
    content = _fetch_content(req_url, filetype)
    _write_files(content, dated_fp, latest_fp)
    meta['last_fetched'] = now.isoformat(timespec='minutes')
    _write_meta(idx_file, meta, body)
    return str(folder.relative_to(base))

def updateData(path: str):
    """Refresh any outdated data sources."""
    base = Path(path)
    now = dt.datetime.now()
    today = now.date()
    updated = []
    for idx_file in base.rglob('index.md'):
        changed = _process_index(idx_file, base, now, today)
        if changed:
            updated.append(changed)
    if updated:
        print('Updated:', ', '.join(updated))
    else:
        print('Everything up to date.')

updateData('./data')


Everything up to date.


## Update Analyses

This cell needs to do the same type of scan across all the analyses in /analysis. It needs to iterate across all the analyses and check the time last modified for all the dependencies. If any dependency was modified more recently than the analysis, then the analysis needs to be run again. The time last modified of the analysis is the most recent file modification time in the analysis directory, because the analysis directory will contain some arbitrary number of output files.  

Because some analyses will list other analyses as dependencies, this loop of checking across all of the analyses needs to keep running until none of them have anything to do, up to some reasonable limit of times to prevent arbitrary recursion.  

In [5]:
from pathlib import Path
import json
import re
import subprocess
import sys
import yaml
from typing import List

def _parse_meta(path: Path):
    """Return metadata and body from a markdown file."""
    text = path.read_text()
    m = re.search(r'^---\n(.*?)\n---\n?', text, re.S)
    if m:
        try:
            meta = yaml.safe_load(m.group(1)) or {}
        except Exception as e:
            print(f'Error parsing metadata in {path}:', e)
            raise
        body = text[m.end():]
    else:
        meta, body = {}, text
    return meta, body

def _latest_mtime(p: Path) -> float:
    """Return the most recent modification time in a path."""
    if p.is_file():
        return p.stat().st_mtime
    mt = [f.stat().st_mtime for f in p.rglob('*') if f.is_file()]
    return max(mt) if mt else p.stat().st_mtime

def build_dep_map(analysis_dir: Path, repo_dir: Path):
    """Map notebooks to their dependency paths."""
    dep_map = {}
    for meta in analysis_dir.rglob('index.md'):
        info, _ = _parse_meta(meta)
        deps = [(repo_dir / d).resolve() for d in info.get('dependencies', [])]
        for nb in meta.parent.glob('*.ipynb'):
            dep_map[nb] = deps
    return dep_map

def outdated(nb: Path, deps: List[Path]) -> bool:
    """Return True if any dependency is newer than the notebook."""
    nb_m = _latest_mtime(nb)
    return any(_latest_mtime(d) > nb_m for d in deps)

def execute(nb: Path):
    """Run a notebook in-place if jupyter is available."""
    import shutil
    if not shutil.which('jupyter'):
        print('jupyter not available - skipping', nb)
        return
    cmd = [
        sys.executable, '-m', 'jupyter', 'nbconvert',
        '--to', 'notebook', '--inplace', '--execute',
        '--ExecutePreprocessor.timeout=600', str(nb),
    ]
    subprocess.run(cmd, check=False)

def updateAnalyses(path: str):
    """Execute notebooks whose dependencies have changed."""
    repo_dir = Path('.').resolve()
    analysis_dir = repo_dir / path
    for _ in range(10):
        dep_map = build_dep_map(analysis_dir, repo_dir)
        outdated_nbs = [
            nb for nb, deps in dep_map.items() if deps and outdated(nb, deps)
        ]
        json.dump(
            {'outdated_notebooks': [str(nb) for nb in outdated_nbs]},
            open('dependencies.json', 'w'),
            indent=2,
        )
        if not outdated_nbs:
            print('Everything up to date.')
            break
        for nb in outdated_nbs:
            execute(nb)

updateAnalyses('./analysis')


KeyboardInterrupt: 